In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Cargar el dataset de los Juegos Olímpicos
df_olimpics = pd.read_csv('athlete_events.csv')
df_olimpics = df_olimpics[['Year', 'Team', 'Sport', 'Medal', 'Sex']]

# Inicializar la app de Dash
app = dash.Dash(__name__)

# Definir el layout de la app
app.layout = html.Div([
    html.H1('Medallas Olímpicas'),

    html.Div([
        html.Label('Seleccionar el País'),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df_olimpics['Team'].unique()],
            value='United States',
            multi=False
        ),
    ]),

    html.Div([
        html.Label('Selecciona el Deporte'),
        dcc.Dropdown(
            id='sport-dropdown',
            options=[{'label': sport, 'value': sport} for sport in df_olimpics['Sport'].unique()],
            multi=False
        ),
    ]),

    dcc.Graph(id='medals-graph'),

    html.Div(id='summary-text')
])

# Definir la lógica de los callbacks
@app.callback(
    [Output('medals-graph', 'figure'),
     Output('summary-text', 'children')],
    [Input('country-dropdown', 'value'),
     Input('sport-dropdown', 'value')]
)
def update_graph(selected_country, selected_sport):
    filtered_df = df_olimpics[(df_olimpics['Team'] == selected_country) & (df_olimpics['Sport'] == selected_sport)]

    # Contar las medallas por tipo
    medals_count = filtered_df['Medal'].value_counts().reset_index()
    medals_count.columns = ['Medal', 'Count']

    # Crear el gráfico de barras
    fig = px.bar(medals_count, x='Medal', y='Count', color='Medal',
                 title=f'Medallas ganadas por {selected_country} en {selected_sport}')

    # Texto resumen
    total_medals = filtered_df.shape[0]
    text_summary = f'{selected_country} ha ganado un total de {total_medals} medallas en {selected_sport}.'

    return fig, text_summary

# Ejecutar la app
if __name__ == '__main__':
    app.run_server(debug=True)